# Job Posting Collection Denormalization Process

1. Benefits Dataset: Contains job_id and the type of benefits offered.
2. Job Industries Dataset: Links job_id with industry_id.
3. Job Postings Dataset: A comprehensive dataset with various details about each job posting, such as job_id, company_id, job title, description, salary information, location, experience level, etc.
4. Job Skills Dataset: Associates job_id with required skill abbreviations (skill_abr).
5. Salaries Dataset: Provides detailed salary information like max_salary, med_salary, min_salary, and the payment period for each job_id.

To denormalize these datasets into a single table, we need to merge them based on the job_id key. The best approach is to start with the Job Postings dataset as the base, as it contains the most comprehensive information, and then sequentially join the other datasets on the job_id column.

In [10]:
import pandas as pd
# Paths to your CSV files
job_skills_path = 'cleaned_job_skills.csv'
job_industries_path = 'cleaned_job_industries.csv'
job_postings_path = 'cleaned_job_postings.csv'
salaries_path = 'cleaned_salaries.csv'
benefits_path = 'cleaned_benefits.csv'

# Read the datasets into pandas DataFrames
job_skills_df = pd.read_csv(job_skills_path)
job_industries_df = pd.read_csv(job_industries_path)
job_postings_df = pd.read_csv(job_postings_path)
salaries_df = pd.read_csv(salaries_path)
benefits_df = pd.read_csv(benefits_path)

# Merging the datasets
# Start by merging job postings with job skills
denormalized_df = pd.merge(job_postings_df, job_skills_df, on='job_id', how='left')

# Merge with job industries
denormalized_df = pd.merge(denormalized_df, job_industries_df, on='job_id', how='left')

# Merge with salaries
denormalized_df = pd.merge(denormalized_df, salaries_df, on='job_id', how='left')

# Merge with benefits
denormalized_df = pd.merge(denormalized_df, benefits_df, on='job_id', how='left')

# Now denormalized_df contains the merged dataset
# You can explore it using denormalized_df.head() or save it to a new CSV
denormalized_df.to_csv('denormalized_job_postings_data.csv', index=False)


In [11]:
denormalized_df.shape[0]

59255

In [12]:
denormalized_df.head()

,job_id,company_id,title,description,max_salary_x,med_salary_x,min_salary_x,pay_period_x,formatted_work_type,location,...,skill_abr,industry_id,salary_id,max_salary_y,med_salary_y,min_salary_y,pay_period_y,currency_y,compensation_type_y,type
0,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,SALE,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medical insurance
1,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,SALE,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401(k)
2,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,SALE,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vision insurance
3,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,BD,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medical insurance
4,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,BD,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401(k)


In [13]:
# Load the datasets
benefits_df = pd.read_csv('cleaned_benefits.csv')
job_industries_df = pd.read_csv('cleaned_job_industries.csv')
job_postings_df = pd.read_csv('cleaned_job_postings.csv')
job_skills_df = pd.read_csv('cleaned_job_skills.csv')
salaries_df = pd.read_csv('cleaned_salaries.csv')

# Displaying the first few rows of each dataset for analysis
benefits_df.head(), job_industries_df.head(), job_postings_df.head(), job_skills_df.head(), salaries_df.head()

(       job_id                  type
 0  3690843087     Medical insurance
 1  3690843087      Dental insurance
 2  3690843087                401(k)
 3  3690843087  Paid maternity leave
 4  3690843087  Disability insurance,
        job_id  industry_id
 0  3378133231           68
 1  3497509795           96
 2  3690843087           47
 3  3691775263          112
 4  3691779379           80,
       job_id  company_id                title  \
 0  133114754  77766802.0        Sales Manager   
 1  133196985   1089558.0   Model Risk Auditor   
 2  381055942  96654609.0     Business Manager   
 3  529257371   1244539.0  NY Studio Assistant   
 4  903408693   3894635.0     Office Associate   
 
                                          description     max_salary  \
 0  Are you a dynamic and creative marketing profe...  Not Available   
 1  Join Us as a Model Risk Auditor  Showcase Your...  Not Available   
 2  Business ManagerFirst Baptist Church ForneyFor...  Not Available   
 3  YOU COULD BE O

In [14]:
# Merging the datasets on 'job_id'
# Using left joins to keep all rows from the job_postings_df and adding information from other datasets

# Merging job_postings_df with salaries_df
merged_df = pd.merge(job_postings_df, salaries_df, on='job_id', how='left', suffixes=('', '_sal'))

# Merging with benefits_df
# Here, we aggregate the benefits as a list to avoid duplication of rows
benefits_agg = benefits_df.groupby('job_id')['type'].apply(list).reset_index()
merged_df = pd.merge(merged_df, benefits_agg, on='job_id', how='left')

# Merging with job_skills_df
# Aggregating skills as well
skills_agg = job_skills_df.groupby('job_id')['skill_abr'].apply(list).reset_index()
merged_df = pd.merge(merged_df, skills_agg, on='job_id', how='left')

# Merging with job_industries_df
# Aggregating industries as well
industries_agg = job_industries_df.groupby('job_id')['industry_id'].apply(list).reset_index()
merged_df = pd.merge(merged_df, industries_agg, on='job_id', how='left')

# Display the first few rows of the merged dataset
merged_df.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,salary_id,max_salary_sal,med_salary_sal,min_salary_sal,pay_period_sal,currency_sal,compensation_type_sal,type,skill_abr,industry_id
0,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Medical insurance, 401(k), Vision insurance]","[SALE, BD]",[92]
1,133196985,1089558.0,Model Risk Auditor,Join Us as a Model Risk Auditor Showcase Your...,Not Available,Not Available,Not Available,not given,Contract,"New York, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ACCT, FIN]",[104]
2,381055942,96654609.0,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,Not Available,Not Available,Not Available,not given,Full-time,"Forney, TX",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[89]
3,529257371,1244539.0,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,Not Available,Not Available,Not Available,not given,Full-time,"New York, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Medical insurance, Vision insurance, Dental i...","[DSGN, ART, IT]",[99]
4,903408693,3894635.0,Office Associate,Provide clerical and administrative support to...,42000.0,Not Available,37000.0,YEARLY,Full-time,"Albany, GA",...,163.0,42000.0,NaN,37000.0,YEARLY,USD,BASE_SALARY,NaN,[ADM],"[27, 34]"


In [15]:
# Getting the number of rows in the merged dataset
num_rows = merged_df.shape[0]
num_rows

15520

In [16]:
# Displaying the first five rows of the merged dataset
merged_df.head()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,salary_id,max_salary_sal,med_salary_sal,min_salary_sal,pay_period_sal,currency_sal,compensation_type_sal,type,skill_abr,industry_id
0,133114754,77766802.0,Sales Manager,Are you a dynamic and creative marketing profe...,Not Available,Not Available,Not Available,not given,Full-time,"Santa Clarita, CA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Medical insurance, 401(k), Vision insurance]","[SALE, BD]",[92]
1,133196985,1089558.0,Model Risk Auditor,Join Us as a Model Risk Auditor Showcase Your...,Not Available,Not Available,Not Available,not given,Contract,"New York, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[ACCT, FIN]",[104]
2,381055942,96654609.0,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,Not Available,Not Available,Not Available,not given,Full-time,"Forney, TX",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[89]
3,529257371,1244539.0,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,Not Available,Not Available,Not Available,not given,Full-time,"New York, NY",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Medical insurance, Vision insurance, Dental i...","[DSGN, ART, IT]",[99]
4,903408693,3894635.0,Office Associate,Provide clerical and administrative support to...,42000.0,Not Available,37000.0,YEARLY,Full-time,"Albany, GA",...,163.0,42000.0,NaN,37000.0,YEARLY,USD,BASE_SALARY,NaN,[ADM],"[27, 34]"


In [17]:
merged_df.to_csv('denormalized_job_postings_data.csv', index=False)